# How to Define Custom Gates
`PauliPropagation.jl` is extensible and allows you to define your own gates. Depending on how much you can or want to code, you can definte a gate that _works_ or one that is as fast as it gets. Here will see what you need to define.

In [1]:
using PauliPropagation

### A static gate that maps one Pauli string to one Pauli string

Let us start by defining a `SWAP` gate. It is sub-typing from `StaticGate`, which denotes that it does not take any variable parameters at propagation time. It always acts the same.

In [2]:
struct CustomSWAPGate <: StaticGate
    qinds::Tuple{Int, Int}  # The two sites to be swapped
end

The action of a `SWAP` gate on a Pauli string is that it swaps the Paulis on two sites. We can now define a function `apply` which receives these 3 arguments in this order, as well as potential `kwargs`: `apply(gate::YourGate, pstr, coeff; kwargs...)`. We can ignore `kwargs` for now, but you can use them to pass arguments from the top level down to your function. If your custom gate is parametrized, then you have you sub-type `ParametrizedGate` instead of `StaticGate`, and the `apply` function
will receive a fourth argument, which is the parameter: `apply(gate::YourGate, pstr, coeff, theta; kwargs...)`.

This is how you can define `SWAP`:

In [3]:
function PauliPropagation.apply(gate::CustomSWAPGate, pstr, coeff; kwargs...)
    # get the Pauli on the first site
    pauli1 = getpauli(pstr, gate.qinds[1])
    # get the Pauli on the second site
    pauli2 = getpauli(pstr, gate.qinds[2])
    
    # set the Pauli on the first site to the second Pauli
    pstr = setpauli(pstr, pauli2, gate.qinds[1])
    # set the Pauli on the second site to the first Pauli
    pstr = setpauli(pstr, pauli1, gate.qinds[2])

    # apply() is always expected to return a tuple of (pstr, coeff) tuples
    return ((pstr, coeff),)
end

This is it, really.

`apply()` is expected to always return a tuple of `(pstr, coeff)` tuples. Here, only one Pauli string and its coefficient are returned, so we need to create a tuple of length one via `((pstr, coeff),)`. Alternatively `tuple((pstr, coeff))` also works. We will see later what to do with gates that create new Pauli strings.

Try the SWAP gate on a few very simple examples:

In [4]:
# define the gate
g = CustomSWAPGate((1, 2))

# define a Pauli string
pstr = PauliString(2, :X, 1)
# turn into a PauliSum so it is easier to compare
psum = PauliSum(pstr)
println("Paulis before: ", psum)

swapped_psum = propagate(g, psum)
println("Paulis after: ", swapped_psum)

Paulis before: PauliSum(nqubits: 2, 1 Pauli term: 
 1.0 * XI
)
Paulis after: PauliSum(nqubits: 2, 1 Pauli term: 
 1.0 * IX
)


In [5]:
nq = 3

# define the gate
g = CustomSWAPGate((2, 3))

# add some initial Pauli strings
psum = PauliSum(nq) 
add!(psum, :X, 1, 0.1)
add!(psum, :X, 2, 0.2)
add!(psum, [:Z, :Z], [1, 2], 0.3)
add!(psum, [:Y, :X], [2, 3], 0.4)
println("Paulis before: ", psum)

swapped_psum = propagate(g, psum)
println("Paulis after: ", swapped_psum)

Paulis before: PauliSum(nqubits: 3, 4 Pauli terms:
 0.2 * IXI
 0.3 * ZZI
 0.4 * IYX
 0.1 * XII
)
Paulis after: PauliSum(nqubits: 3, 4 Pauli terms:
 0.3 * ZIZ
 0.2 * IIX
 0.4 * IXY
 0.1 * XII
)


This works! You can now freely plug such a gate into a circuit and let it swap your qubits.

Consider a simple more wholistic example where we insert a layer of SWAP gates at the beginning of the circuit, which means at the end of the backpropagation.

In [6]:
nq = 6

# some initial observable
pstr = PauliString(nq, [:X, :X], [3, 4])

# a 1D bricklayer topology
topology = bricklayertopology(nq)

# an empty circuit
circuit = Gate[]
# an RX Pauli rotation layer
rxlayer!(circuit, nq)
# an RZ Pauli rotation layer
rzlayer!(circuit, nq)
# an RZZ Pauli rotation layer on the topology 
rzzlayer!(circuit, topology);


# define some parameters
using Random
Random.seed!(420)
nparams = countparameters(circuit)
thetas = randn(nparams);

Now define our custom layer of SWAP gates:

In [7]:
ourSWAPs = Gate[]
# insert swaps flipping the order of qubits 4 to 6
# 4 5 6 -> 5 4 6
push!(ourSWAPs, CustomSWAPGate((4, 5)))
# 5 4 6 -> 5 6 4
push!(ourSWAPs, CustomSWAPGate((5, 6)))
# 5 6 4 -> 6 5 4
push!(ourSWAPs, CustomSWAPGate((4, 5)));

In [8]:
# this flattens the two circuits into one
ourSWAP_circuit = [ourSWAPs..., circuit...]

20-element Vector{Gate}:
 CustomSWAPGate((4, 5))
 CustomSWAPGate((5, 6))
 CustomSWAPGate((4, 5))
 PauliRotation([:X], [1])
 PauliRotation([:X], [2])
 PauliRotation([:X], [3])
 PauliRotation([:X], [4])
 PauliRotation([:X], [5])
 PauliRotation([:X], [6])
 PauliRotation([:Z], [1])
 PauliRotation([:Z], [2])
 PauliRotation([:Z], [3])
 PauliRotation([:Z], [4])
 PauliRotation([:Z], [5])
 PauliRotation([:Z], [6])
 PauliRotation([:Z, :Z], [1, 2])
 PauliRotation([:Z, :Z], [3, 4])
 PauliRotation([:Z, :Z], [5, 6])
 PauliRotation([:Z, :Z], [2, 3])
 PauliRotation([:Z, :Z], [4, 5])

Propagate through the circuit and overlap with the zero state:

In [9]:
our_psum = propagate(ourSWAP_circuit, pstr, thetas)

PauliSum(nqubits: 6, 81 Pauli terms:
 -0.18621 * IZZIIY
 -0.022844 * IZXIZZ
 0.095416 * IYXIIX
 -0.056601 * IIZIIY
 -0.060448 * IYZIYY
 0.016621 * IYYIIY
 0.010874 * IZYIYY
 0.0090624 * IZYIZY
 0.071515 * IZZIZY
 -0.0071724 * IIYIIY
 -0.018666 * IYYIIZ
 0.13117 * IYZIIY
 -0.00766 * IYYIYY
 0.020342 * IZXIZY
 -0.017985 * IZXIZX
 0.042508 * IYYIIX
 0.0033054 * IIYIYY
 0.019309 * IYXIYZ
 -0.023063 * IIZIYX
 0.026085 * IIZIYY
  ⋮)

In [10]:
overlapwithzero(our_psum)

0.16795162488187168

This looks okay, but is it correct? One thing you may have noticed is that `SWAP` is a `Clifford` operation, i.e., one that takes one Pauli to exactly one other Pauli. We actually have that in our package so we can easily compare.

In [11]:
cliffordSWAPs = Gate[]
# insert swaps flipping the order of qubits 4 to 6
# 4 5 6 -> 5 4 6
push!(cliffordSWAPs, CliffordGate(:SWAP, (4, 5)))
# 5 4 6 -> 5 6 4
push!(cliffordSWAPs, CliffordGate(:SWAP, (5, 6)))
# 5 6 4 -> 6 5 4
push!(cliffordSWAPs, CliffordGate(:SWAP, (4, 5)));

In [12]:
# this flattens the two circuits into one
cliffordSWAP_circuit = [cliffordSWAPs..., circuit...]

20-element Vector{Gate}:
 CliffordGate(:SWAP, [4, 5])
 CliffordGate(:SWAP, [5, 6])
 CliffordGate(:SWAP, [4, 5])
 PauliRotation([:X], [1])
 PauliRotation([:X], [2])
 PauliRotation([:X], [3])
 PauliRotation([:X], [4])
 PauliRotation([:X], [5])
 PauliRotation([:X], [6])
 PauliRotation([:Z], [1])
 PauliRotation([:Z], [2])
 PauliRotation([:Z], [3])
 PauliRotation([:Z], [4])
 PauliRotation([:Z], [5])
 PauliRotation([:Z], [6])
 PauliRotation([:Z, :Z], [1, 2])
 PauliRotation([:Z, :Z], [3, 4])
 PauliRotation([:Z, :Z], [5, 6])
 PauliRotation([:Z, :Z], [2, 3])
 PauliRotation([:Z, :Z], [4, 5])

In [13]:
clifford_psum = propagate(cliffordSWAP_circuit, pstr, thetas)

PauliSum(nqubits: 6, 81 Pauli terms:
 -0.18621 * IZZIIY
 -0.022844 * IZXIZZ
 0.095416 * IYXIIX
 -0.056601 * IIZIIY
 -0.060448 * IYZIYY
 0.016621 * IYYIIY
 0.010874 * IZYIYY
 0.0090624 * IZYIZY
 0.071515 * IZZIZY
 -0.0071724 * IIYIIY
 -0.018666 * IYYIIZ
 0.13117 * IYZIIY
 -0.00766 * IYYIYY
 0.020342 * IZXIZY
 -0.017985 * IZXIZX
 0.042508 * IYYIIX
 0.0033054 * IIYIYY
 0.019309 * IYXIYZ
 -0.023063 * IIZIYX
 0.026085 * IIZIYY
  ⋮)

And these produce the same Pauli sum and consequently the same expectation!

In [14]:
our_psum == clifford_psum

true

In [15]:
overlapwithzero(clifford_psum)

0.16795162488187168

### A parametrized custom noise channel
The above example was not only a Clifford gate mapping one Pauli string to one Pauli string, but it was also not parametrized. We will now implement a funky custom and parametrized noise gate. Note that it may technically not be a valid quantum channel, but we can still define it and have it be part of the simulation.

In [16]:
struct FunkyNoise <: ParametrizedGate
    qinds::Tuple{Int, Int}  # say it acts on 2 qubits
end

Say we want to define a noise gate with strength `p` that behaves in the following way:
```
XX -> (1 - 2p) * XX + p * IX + p * XI
YY -> (1 - 2p) * YY + p * IY + p * YI
ZZ -> (1 - 2p) * ZZ + p * IZ + p * ZI
P -> P else
```
Let's now implement that.

In [17]:
# if your gate is a subtype of `ParametrizedGate`, `apply()` receives a fourth argument which is the parameter
function PauliPropagation.apply(gate::FunkyNoise, pstr, coeff, p; kwargs...)
    
    # the integer representation of the Paulis sitting on the active sites
    paulis = getpauli(pstr, gate.qinds)

    # check whether the Paulis in `paulis` are the integer representations of our targets
    # "|" is the OR operator
    # if you replaced the (:X, :X), (:Y, :Y), or (:Z, :Z) by their integer representation,
    # i.e., 5, 10, or 15, respectively, it would be faster 
    if ispauli(paulis, (:X, :X)) | ispauli(paulis, (:Y, :Y)) | ispauli(paulis, (:Z, :Z))
        
        # calculate the coefficients
        coeff_PP = coeff * (1 - 2p)
        coeff_IP = coeff_PI = coeff * p        

        # create new Paulis that have I in the right position
        # if you replace :I with the integer 0, it will be faster
        pstr_IP = setpauli(pstr, :I, gate.qinds[1])
        pstr_PI = setpauli(pstr, :I, gate.qinds[2])

        return ((pstr, coeff_PP), (pstr_IP, coeff_IP), (pstr_PI, coeff_PI))
    end

    # else do nothing and return the original pstr and coefficient pair
    return ((pstr, coeff),)
end

A was a bit more challenging than above, but we are done!

In [18]:
nq = 2
g = FunkyNoise((1, 2))
p = 0.1

0.1

In [19]:
# this branches
pstr = PauliString(nq, [:X, :X], [1, 2])
propagate(g, pstr, p)

PauliSum(nqubits: 2, 3 Pauli terms:
 0.8 * XX
 0.1 * IX
 0.1 * XI
)

In [20]:
# and this branches
pstr = PauliString(nq, [:Y, :Y], [1, 2])
propagate(g, pstr, p)

PauliSum(nqubits: 2, 3 Pauli terms:
 0.1 * YI
 0.8 * YY
 0.1 * IY
)

In [21]:
# and this branches
pstr = PauliString(nq, [:Z, :Z], [1, 2])
propagate(g, pstr, p)

PauliSum(nqubits: 2, 3 Pauli terms:
 0.8 * ZZ
 0.1 * IZ
 0.1 * ZI
)

In [22]:
# but not this
pstr = PauliString(nq, [:Y, :Z], [1, 2])
propagate(g, pstr, p)

PauliSum(nqubits: 2, 1 Pauli term: 
 1.0 * YZ
)

In [23]:
# or this
pstr = PauliString(nq, [:I, :X], [1, 2])
propagate(g, pstr, p)

PauliSum(nqubits: 2, 1 Pauli term: 
 1.0 * IX
)

In [24]:
# or this
pstr = PauliString(nq, [:I, :I], [1, 2])
propagate(g, pstr, p)

PauliSum(nqubits: 2, 1 Pauli term: 
 1.0 * II
)

This is how you can implement custom gates - by defining their action on Pauli strings (in the Heisenberg picture).

The latter custom noise gate example especially could be implemented more efficiently via some lower level details, but this certainly works. This way of defining `apply()` for a gate that either branches into three or remains as one Pauli string is also usually not optimal because the compiler cannot infer from the input types whether the output will be a length 3 tuple or a length 1 tuple. This is related to type-stability ([some Julia docs](https://docs.julialang.org/en/v1/manual/performance-tips/#Write-%22type-stable%22-functions)) and a topic of a more advanced example notebook.